<a href="https://colab.research.google.com/github/JSJeong-me/Python_RPA/blob/main/amazon-de.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import re

In [2]:
def cleanText(readData):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', readData)
    return text

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0',
    'Accept-Language': 'en-US, en;q=0.5'
}

In [4]:
search_query = 'luftreiniger'.replace(' ', '+') # zep
base_url = 'https://www.amazon.de/s?k={0}'.format(search_query)

In [5]:
items = []
for i in range(1, 100):
    print('Processing {0}...'.format(base_url + '&page={0}'.format(i)))
    response = requests.get(base_url + '&page={0}'.format(i), headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    results = soup.find_all('div', {'class': 's-result-item', 'data-component-type': 's-search-result'})

    for result in results:
        product_name = result.h2.text

        try:
            rating = result.find('i', {'class': 'a-icon'}).text
            rating_count = result.find_all('span', {'aria-label': True})[1].text
        except AttributeError:
            continue

        try:
            price1 = result.find('span', {'class': 'a-price-whole'}).text
            price1 = cleanText(price1)
            # price2 = result.find('span', {'class': 'a-price-fraction'}).text
            # price2 = cleanText(price2)
            # price = float(price1 + price2)
            price = float(price1)
            product_url = 'https://amazon.de' + result.h2.a['href']
            # print(rating_count, product_url)
            items.append([product_name, rating, rating_count, price, product_url])
        except AttributeError:
            continue
    sleep(1.5)

Processing https://www.amazon.de/s?k=luftreiniger&page=1...
Processing https://www.amazon.de/s?k=luftreiniger&page=2...
Processing https://www.amazon.de/s?k=luftreiniger&page=3...
Processing https://www.amazon.de/s?k=luftreiniger&page=4...
Processing https://www.amazon.de/s?k=luftreiniger&page=5...
Processing https://www.amazon.de/s?k=luftreiniger&page=6...
Processing https://www.amazon.de/s?k=luftreiniger&page=7...
Processing https://www.amazon.de/s?k=luftreiniger&page=8...
Processing https://www.amazon.de/s?k=luftreiniger&page=9...
Processing https://www.amazon.de/s?k=luftreiniger&page=10...
Processing https://www.amazon.de/s?k=luftreiniger&page=11...
Processing https://www.amazon.de/s?k=luftreiniger&page=12...
Processing https://www.amazon.de/s?k=luftreiniger&page=13...
Processing https://www.amazon.de/s?k=luftreiniger&page=14...
Processing https://www.amazon.de/s?k=luftreiniger&page=15...
Processing https://www.amazon.de/s?k=luftreiniger&page=16...
Processing https://www.amazon.de/

In [6]:
df = pd.DataFrame(items, columns=['product', 'rating', 'rating count', 'price', 'product url'])
df['price'] = df['price']/100

In [7]:
df.to_csv('amazon-de-06-{0}.csv'.format(search_query), index=False)

In [8]:
df.tail()

,product,rating,rating count,price,product url
52,AINUORENYI Replacement HEPA Replacement Filter...,3.0 out of 5 stars,1,38.99,https://amazon.de/-/en/AINUORENYI-Replacement-...
53,"LiRiQi Air Purifier HEPA Filter, Fine Dust and...",2.4 out of 5 stars,2,19.99,https://amazon.de/-/en/Purifier-Allergens-Ligh...
54,"AIRROBO AR400 Original Replacement Filter, H13...",4.0 out of 5 stars,1,25.99,https://amazon.de/-/en/Replacement-Efficient-A...
55,NERIOX CLEAN-AIR Mobile Antiviral Air Purifier...,5.0 out of 5 stars,3,712.80,https://amazon.de/-/en/NERIOX-CLEAN-AIR-Antivi...
56,TOPPIN Air Purifier with H13 HEPA Filter TPAP0...,4.6 out of 5 stars,"1,301",49.00,https://amazon.de/-/en/TOPPIN-Purifier-TPAP001...
